# Analyzing Historical Stock Data: Tesla and GameStop Stock Price vs Revenue Analysis

This notebook focuses on extracting and visualizing stock data for Tesla and GameStop. 
We'll use Python libraries such as yfinance, pandas, and plotly to:

1. Fetch historical stock price data
2. Extract revenue data through web scraping
3. Create interactive visualizations comparing stock prices and revenue

The analysis will provide insights into the historical performance of these two high-profile stocks, 
demonstrating key data science skills including data acquisition, cleaning, and visualization.

## Table of Contents

- [Visualisation](#define-graphing-function)
- [Tesla Stock Data](#tesla-stock-data)
- [Webscraping to Extract Tesla Revenue Data](#webscraping-to-extract-tesla-revenue-data)
- [GME Extract Stock Data](#gme-stock-data)
- [Extract GME Revenue Data](#webscraping-to-extract-gme-revenue-data)
- [Tesla Stock Graph](#plot-tesla-stock-graph)
- [GameStop Stock Graph](#plot-gamestop-stock-graph)

In [1]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
import warnings
# Ignore all warnings
warnings.filterwarnings("ignore", category=FutureWarning)

## Define Graphing Function


In [3]:
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    stock_data_specific = stock_data[stock_data.Date <= '2021--06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date), y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date), y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()

## Tesla Stock Data


In [4]:
tesla = yf.Ticker("TSLA")

tesla_data = tesla.history(period="max")

tesla_data.reset_index(inplace=True)

tesla_data['Date'] = tesla_data['Date'].apply(pd.to_datetime)

tesla_data.head()


,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2010-06-29 00:00:00-04:00,1.266667,1.666667,1.169333,1.592667,281494500,0.0,0.0
1,2010-06-30 00:00:00-04:00,1.719333,2.028000,1.553333,1.588667,257806500,0.0,0.0
2,2010-07-01 00:00:00-04:00,1.666667,1.728000,1.351333,1.464000,123282000,0.0,0.0
3,2010-07-02 00:00:00-04:00,1.533333,1.540000,1.247333,1.280000,77097000,0.0,0.0
4,2010-07-06 00:00:00-04:00,1.333333,1.333333,1.055333,1.074000,103003500,0.0,0.0


## Tesla Revenue Data


In [5]:
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm"

html_data = requests.get(url).text

soup = BeautifulSoup(html_data, 'html.parser')

tesla_revenue = pd.DataFrame(columns=["Date", "Revenue"])

for table in soup.find_all("table"):
    
    if table.find("th").__contains__("Tesla Quarterly Revenue"):
        
        for row in table.find("tbody").find_all("tr"):
            col = row.find_all("td")
            date = col[0].text
            revenue = col[1].text
            
            tesla_revenue = pd.concat([tesla_revenue, pd.DataFrame({"Date":[date], "Revenue": revenue})], ignore_index=True)

        break

tesla_revenue["Revenue"] = tesla_revenue['Revenue'].str.replace(',|\$',"", regex=True)
    
tesla_revenue.dropna(inplace=True)

tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] != ""]

tesla_revenue["Revenue"] = tesla_revenue['Revenue'].astype(float)
tesla_revenue["Date"] = tesla_revenue['Date'].apply(pd.to_datetime)

make_graph(tesla_data, tesla_revenue, "Tesla")

## GME Stock Data


Using the `Ticker` function enter the ticker symbol of the stock we want to extract data on to create a ticker object. The stock is GameStop and its ticker symbol is `GME`.


In [6]:
gme = yf.Ticker("GME")

gme_data = gme.history(period="max")

gme_data.reset_index(inplace=True)

gme_data['Date'] = gme_data['Date'].apply(pd.to_datetime)

## GME Revenue Data


In [7]:
url_2 = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html"

html_data_2 = requests.get(url_2).text

soup_2 = BeautifulSoup(html_data_2, 'html.parser')

soup_2.find_all("table")

gme_revenue = pd.DataFrame(columns=["Date", "Revenue"])

for table in soup_2.find_all("table"):
    
    if table.find("th").__contains__("GameStop Quarterly Revenue"):
        
        for row in table.find("tbody").find_all("tr"):
            col = row.find_all("td")
            date = col[0].text
            revenue = col[1].text
            
            gme_revenue = pd.concat([gme_revenue, pd.DataFrame({"Date":[date], "Revenue": revenue})], ignore_index=True)

        break

gme_revenue["Revenue"] = gme_revenue['Revenue'].str.replace(',|\$',"", regex=True)

gme_revenue = gme_revenue[gme_revenue['Revenue'] != ""]

gme_revenue["Revenue"] = gme_revenue['Revenue'].astype(float)
gme_revenue["Date"] = gme_revenue['Date'].apply(pd.to_datetime)

gme_revenue.dropna(inplace=True)

make_graph(gme_data, gme_revenue, "GameStop")